In [2]:
from bs4 import BeautifulSoup
import requests
import json

## Get full html document

In [20]:
url = 'https://www.yelp.com/search?find_desc=bakeries&find_loc=Avigliana,+Italy&start=70'
html_text = requests.get(url, proxies = {"socks5":"http://108.61.175.7:31802"})

## Using BeautifulSoup parsing the html document

In [22]:
print(html_text)

<Response [503]>


In [26]:
response = requests.get('http://jsonip.com', proxies={"socks5":"https://108.61.175.7:31802"})
ip = response.json()['ip']
print('Your public IP is:', ip)

Your public IP is: 76.168.3.83


In [30]:
s = requests.Session()
s.proxies={'http':"socks5://108.61.175.7:31802"}
r = s.get('http://jsonip.com')
print('Your public IP is:', ip)

ConnectTimeout: SOCKSHTTPConnectionPool(host='jsonip.com', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.contrib.socks.SOCKSConnection object at 0x000002040C342490>, 'Connection to jsonip.com timed out. (connect timeout=None)'))

In [23]:
soup = BeautifulSoup(html_text.content,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html class="ie6 ie ltie9 ltie8 no-js" lang="en-US"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie7 ie ltie9 ltie8 no-js" lang="en-US"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie8 ie ltie9 no-js" lang="en-US"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie9 ie no-js" lang="en-US"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="en-US">
 <!--<![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/error_pages/81b13bc440cd/assets/scss/maintenance-pkg.css" rel="stylesheet"/>
  <title>
   Yelp
  </title>
  <link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_styleguide/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/>
  <link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_styleguide/dcfe403147fc/asse

## Get the restaurant list

In [25]:
# div
# class="arrange-unit__09f24__rqHTg arrange-unit-fill__09f24__CUubG border-color--default__09f24__NPAKY"

data = soup.find_all('script',attrs={ "data-hypernova-key" :"yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" })#  attrs={ "data-hypernova-key" :"yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" }
data

[<script data-hypernova-id="d292799d-d4b8-44f2-a340-19d86b22a55d" data-hypernova-key="yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" type="application/json"><!--{"locale":"en_US","messages":{},"requestCookies":{"LOCATION":null,"MO_MAP":null},"requestHeaders":{},"requestPath":"/search","requestUrl":"https://www.yelp.com/search?find_desc=Burgers&amp;find_loc=San+Francisco%2C+CA&amp;start=0","requestTimestampMs":1695444115473,"routeParams":{},"userId":null,"bizUserId":null,"uniqueRequestId":"ce6ee596377d78a6","site":"www","gaConfig":{"clientID":"724C5F8ACEA08AA0","dimensions":{"global":{"full_url":[15,"/search?find_desc=Burgers&amp;find_loc=San+Francisco%2C+CA&amp;start=0"],"internal_ip":[18,false],"referrer":[28,"none"]},"www":{"full_url":[34,"/search?find_desc=Burgers&amp;find_loc=San+Francisco%2C+CA&amp;start=0"],"internal_ip":[27,false],"referrer":[64,"none"]}},"domain":"yelp.com","enable_high_volume_events":false,"ga_enabled":true,"js_dimensions":{"global":{},"www":{}},"m

In [51]:
txtstr = data[0].getText()[4:-4]

In [84]:
aaaa=[m.start() for m in re.finditer('{"ranking":', txtstr)]
aa=[m.start() for m in re.finditer(',"scrollablePhotos":', txtstr)]


In [93]:
z=6
txtstr[aaaa[z]:aa[z]]

'{"ranking":7,"isAd":false,"renderAdInfo":false,"name":"Adam’s Grub Shack","alternateNames":[],"businessUrl":"/biz/adam-s-grub-shack-san-francisco?osq=Burgers","categories":[{"title":"Food Trucks","url":"/search?find_desc=Food+Trucks&amp;find_loc=San+Francisco%2C+CA"},{"title":"Burgers","url":"/search?find_desc=Burgers&amp;find_loc=San+Francisco%2C+CA"},{"title":"Sandwiches","url":"/search?find_desc=Sandwiches&amp;find_loc=San+Francisco%2C+CA"}],"priceRange":"","rating":4.5,"isClickableReview":false,"reviewCount":6,"formattedAddress":"","neighborhoods":["Mission"],"phone":"(415) 559-5636","serviceArea":null,"parentBusiness":null,"servicePricing":null,"serviceOfferings":[],"businessAttributes":{},"alias":"adam-s-grub-shack-san-francisco","website":{"href":"http://www.adamsgrubtruck.com","rel":"noopener"}}'

In [19]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import asyncio
import aiohttp
import time

headers={
    User-Agent': "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}



async def main(urls,startnum,ix):
    async with aiohttp.ClientSession(trust_env=True) as session:
        ret = await asyncio.gather(*[get(url,session,startnum+i,ix) for i,url in enumerate(urls)])
        print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
        return ret
async def get(url, session,startnum,i):
    #print(url)
    try:
        async with session.get(url=url,headers=headers,timeout=12) as response:
            resp = await response.read()
            print("{}: Successfully got url {} with resp of length {}.".format(startnum,url, len(resp)))
            if(resp is not None):
                try:
                    soup = BeautifulSoup(resp.content,'lxml')
                    data = soup.find_all('script',attrs={ "data-hypernova-key" :"yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" })#  attrs={ "data-hypernova-key" :"yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" }
                    datastr = data[0].getText()[4:-4]
                    arridx=[m.start() for m in re.finditer('{"ranking":', datastr)]
                    arridy=[m.start() for m in re.finditer(',"scrollablePhotos":', datastr)]
                    loaded = []
                    for z in range(10):
                        loaded.append(json.loads(txtstr[arridx[z]:arridy[z]]))
                        if loaded[z]['website'] is not None:
                            loaded[z]['website']=loaded[z]['website']['href']
                    df = pd.DataFrame(loaded)
                    return df
                    # df.to_csv("data.csv",mode='a')        
                except:
                    i.write(url+'\n')
                    pass

    except Exception as e:
        try:
            print("{}: Unable to get url {} due to {}.".format(startnum,url, e.__class__))
            i.write(url+'\n')
        #        errors.add(url)
            return None
        except:
            print("error")
def chonk(arrdata,chonksize):
    chonk = []
    chonkytonk = []
    count = 0
    for x in arrdata:
        chonk.append(x)
        #print(x)
        if count>chonksize:
            count = 0
            chonkytonk.append(chonk)
            chonk = []
        count = count +1
    return chonkytonk

def main():
    f = open("cities.txt","r")
    cities = f.read().split("\n")
    f.close()
    btype = ["bakeries","cafe"]
    baseurl = 'https://www.yelp.com/search?find_desc='
    locurl = '&find_loc='
    #https://www.yelp.com/search?find_desc=cafe&find_loc=toyko+japan
    start_urls=[]
    start_index = 0
    for i in range(len(cities)):
        for b in btype:
            start_urls.append(baseurl+str(b)+str(locurl)+cities[i])
            for c in range(1,24):
                start_urls.append(baseurl+str(b)+str(locurl)+str(cities[i])+"&start="+str(c*10))
    # chonk = []
    # chonkytonk = []
    # chonksize = 10
    # count = 0
    # for x in start_urls:
    #     chonk.append(x)
    #     #print(x)
    #     if count>chonksize:
    #         count = 0
    #         chonkytonk.append(chonk)
    #         chonk = []
    #     count = count +1
    chonkytonk = chonk(start_urls,10)
    for chunk in chonkytonk:
        i = open("yelp_error.txt","a",encoding="utf-8")
        df = asyncio.run(main(chunk,start_index,i))
        i.close()
        for x in df:
            x.to_csv("data.csv",mode='a')
        del df







https://www.earthtonecafe.com
http://www.themillsf.com
https://breadbellysf.com
http://www.diamondbeans.com
https://thirdculturebakery.com
http://homecoffeesf.com
https://www.noecafe.com
http://www.homecoffeesf.com
http://thecoffeemovement.com


TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [20]:
# url = 'https://www.yelp.com/search?find_desc=Cafe&find_loc=San+Francisco%2C+CA&start=0'
html_text = requests.get(url)

#print(soup.prettify())
soup = BeautifulSoup(html_text.content,'lxml')
data = soup.find_all('script',attrs={ "data-hypernova-key" :"yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" })#  attrs={ "data-hypernova-key" :"yelpfrontend__54515__yelpfrontend__GondolaSearch__dynamic" }
txtstr = data[0].getText()[4:-4]

arridx = re.finditer('{"ranking":', txtstr)
arride = re.finditer(',"scrollablePhotos":',txtstr)
aaaa=[m.start() for m in re.finditer('{"ranking":', txtstr)]
aa=[m.start() for m in re.finditer(',"scrollablePhotos":', txtstr)]

loaded = []
for z in range(10):
    loaded.append(json.loads(txtstr[aaaa[z]:aa[z]]))
    if loaded[z]['website'] is not None:
        loaded[z]['website']=loaded[z]['website']['href']
        print(loaded[z]['website'])

df = pd.DataFrame(loaded)
df.add(pd.DataFrame(loaded))

https://www.earthtonecafe.com
http://www.themillsf.com
https://breadbellysf.com
http://www.diamondbeans.com
https://thirdculturebakery.com
http://homecoffeesf.com
https://www.noecafe.com
http://www.homecoffeesf.com
http://thecoffeemovement.com


TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [74]:
datax = json.loads(txtstr[47693:48402])
datax

{'ranking': 1,
 'isAd': False,
 'renderAdInfo': False,
 'name': 'Uncle Boy’s',
 'alternateNames': [],
 'businessUrl': '/biz/uncle-boys-san-francisco?osq=Burgers',
 'categories': [{'title': 'Burgers',
   'url': '/search?find_desc=Burgers&amp;find_loc=San+Francisco%2C+CA'},
  {'title': 'Fast Food',
   'url': '/search?find_desc=Fast+Food&amp;find_loc=San+Francisco%2C+CA'}],
 'priceRange': '$$',
 'rating': 3.9,
 'isClickableReview': False,
 'reviewCount': 766,
 'formattedAddress': '',
 'neighborhoods': ['Inner Richmond'],
 'phone': '(415) 742-4468',
 'serviceArea': None,
 'parentBusiness': None,
 'servicePricing': None,
 'serviceOfferings': [],
 'businessAttributes': {},
 'alias': 'uncle-boys-san-francisco',
 'website': {'href': 'https://uncleboys.com', 'rel': 'noopener nofollow'}}

In [76]:
datax['website']['href']

'https://uncleboys.com'

In [46]:
json.loads(data[0].getText()[4:-4])

JSONDecodeError: Expecting ',' delimiter: line 1 column 141324 (char 141323)

## Restaurant Name & user responses

In [ ]:
restaurant_names = []
user_responses = []


for item in burgers:
  restaurant_name = item.find('a', class_ = 'css-19v1rkv')
  user_response = item.find('span', class_ = 'css-chan6m')

  if restaurant_name != None:
    restaurant_names.append(restaurant_name.text.strip())

  if user_response != None:
    user_responses.append(user_response.text.strip())

In [ ]:
print(len(restaurant_names))
print(restaurant_names)

10
['Uncle Boy’s', 'Native Burger', 'WesBurger ‘N’ More', 'Roam Artisan Burgers', 'Super Duper Burgers', 'Flippin Burger', 'Hi-Way Burger & Fry', 'The Butcher Shop by Niku Steakhouse', 'Calibur', 'Hidden Spot EXL']


In [ ]:
print(len(user_responses))
print(user_responses)

10
['746', '321', '736', '1352', '1430', '88', '282', '220', '522', '22']


## Location

In [ ]:
services = []
for item in data:
  service = item.find_all('span', class_ = 'css-chan6m')

  if service != []:
    services.append(service)


In [ ]:
for i in services:
  print(i)

[<span class="css-chan6m">746</span>, <span class="css-chan6m">Inner Richmond</span>]
[<span class="css-chan6m">321</span>, <span class="css-chan6m">Laurel Heights</span>]
[<span class="css-chan6m">736</span>, <span class="css-chan6m">Mission</span>]
[<span class="css-chan6m">1352</span>, <span class="css-chan6m">Lower Pacific Heights</span>]
[<span class="css-chan6m">1430</span>, <span class="css-chan6m">Duboce Triangle</span>]
[<span class="css-chan6m">88</span>, <span class="css-chan6m">The Haight</span>]
[<span class="css-chan6m">282</span>, <span class="css-chan6m">Noe Valley</span>]
[<span class="css-chan6m">220</span>, <span class="priceRange__09f24__mmOuH css-chan6m">$$$</span>, <span class="css-chan6m"></span>]
[<span class="css-chan6m">522</span>, <span class="css-chan6m">West Portal</span>]
[<span class="css-chan6m">22</span>, <span class="css-chan6m">Mission Terrace</span>]


In [ ]:
locations = []
for i in services:
  
  locations.append(i[1].text.strip())

In [ ]:
print(len(locations))
print(locations)

10
['Inner Richmond', 'Laurel Heights', 'Mission', 'Lower Pacific Heights', 'Duboce Triangle', 'The Haight', 'Noe Valley', '$$$', 'West Portal', 'Mission Terrace']


##Menu

In [ ]:
food_items = []
for item in data:
  menu = item.find_all('span', class_ = 'css-11bijt4')

  if menu != []:
    list_menu = []
    for i in menu:
      list_menu.append(i.text.strip())
    food_items.append(list_menu)

In [ ]:
print(len(food_items))
print(food_items)

10
[['Burgers', 'Fast Food'], ['Burgers'], ['Burgers'], ['Burgers'], ['Burgers', 'American (Traditional)'], ['Burgers', 'Hot Dogs', 'Sandwiches'], ['Burgers', 'Fast Food'], ['Butcher'], ['Burgers', 'American (Traditional)'], ['Burgers', 'Sandwiches', 'Chicken Wings']]


In [ ]:
original_list = food_items
new_list = [[item for item in sublist] for sublist in original_list]

output_list = []
for sublist in new_list:
    output_list.append(" ".join(sublist))

print(output_list)


['Burgers Fast Food', 'Burgers', 'Burgers', 'Burgers', 'Burgers American (Traditional)', 'Burgers Hot Dogs Sandwiches', 'Burgers Fast Food', 'Butcher', 'Burgers American (Traditional)', 'Burgers Sandwiches Chicken Wings']


##Number of Food Items

In [ ]:
num_items = []
for i in food_items:
  num_items.append(len(i))


In [ ]:
print(len(num_items))
print(num_items)

10
[2, 1, 1, 1, 2, 3, 2, 1, 2, 3]


##Mounting G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Dataset Visualization

In [ ]:
import pandas as pd
data ={
    'Restaurant_name ' : restaurant_names,
    ' Number_of_User_Response ' : user_responses,
    ' Restaurant_Location ' : locations,
    ' Restaurant_Menu ' : output_list ,
    ' Number_of_Food_Items ' : num_items,
}
dataset = pd.DataFrame.from_dict(data)
dataset.head(10)

,Restaurant_name,Number_of_User_Response,Restaurant_Location,Restaurant_Menu,Number_of_Food_Items
0,Uncle Boy’s,746,Inner Richmond,Burgers Fast Food,2
1,Native Burger,321,Laurel Heights,Burgers,1
2,WesBurger ‘N’ More,736,Mission,Burgers,1
3,Roam Artisan Burgers,1352,Lower Pacific Heights,Burgers,1
4,Super Duper Burgers,1430,Duboce Triangle,Burgers American (Traditional),2
5,Flippin Burger,88,The Haight,Burgers Hot Dogs Sandwiches,3
6,Hi-Way Burger & Fry,282,Noe Valley,Burgers Fast Food,2
7,The Butcher Shop by Niku Steakhouse,220,$$$,Butcher,1
8,Calibur,522,West Portal,Burgers American (Traditional),2
9,Hidden Spot EXL,22,Mission Terrace,Burgers Sandwiches Chicken Wings,3


##Saving Scraped Data to Google Drive 

In [ ]:
filename = "yelp_data.csv"
filepath = "/content/drive/MyDrive/Webscraping/YelpScraper/"
df = pd.DataFrame(data)
df.to_csv(filepath + filename, index=False, encoding='utf-8')